# Introduction

So far, you've learned how to use several SQL clauses.  For instance, you know how to use **SELECT** to pull specific columns from a table, along with **WHERE** to pull rows that meet specified criteria.  You also know how to use aggregate functions like **COUNT()**, along with **GROUP BY** to treat multiple rows as a single group.  

Now you'll learn how to change the order of your results using the **ORDER BY** clause, and you'll explore a popular use case by applying ordering to dates.  To illustrate what you'll learn in this tutorial, we'll work with a slightly modified version of our familiar `pets` table.

![](https://i.imgur.com/b99zTLv.png)

# ORDER BY

**ORDER BY** is usually the last clause in your query, and it sorts the results returned by the rest of your query.

Notice that the rows are not ordered by the `ID` column.  We can quickly remedy this with the query below.

![](https://i.imgur.com/6o9LuTA.png)

The **ORDER BY** clause also works for columns containing text, where the results show up in alphabetical order.

![](https://i.imgur.com/ooxuzw3.png)

You can reverse the order using the **DESC** argument (short for 'descending').  The next query sorts the table by the `Animal` column, where the values that are last in alphabetic order are returned first.

![](https://i.imgur.com/IElLJrR.png)
 
# Dates

Next, we'll talk about dates, because they come up very frequently in real-world databases. There are two ways that dates can be stored in BigQuery: as a **DATE** or as a **DATETIME**. 

The **DATE** format has the year first, then the month, and then the day. It looks like this:

```
YYYY-[M]M-[D]D
```
    
* `YYYY`: Four-digit year
* `[M]M`: One or two digit month
* `[D]D`: One or two digit day

So `2019-01-10` is interpreted as January 10, 2019.

The **DATETIME** format is like the date format ... but with time added at the end.

# EXTRACT

Often you'll want to look at part of a date, like the year or the day. You can do this with **EXTRACT**.  We'll illustrate this with a slightly different table, called `pets_with_date`.

![](https://i.imgur.com/vhvHIh0.png)

The query below returns two columns, where column `Day` contains the day corresponding to each entry the `Date` column from the `pets_with_date` table: 
            
![](https://i.imgur.com/PhoWBO0.png)

SQL is very smart about dates, and we can ask for information beyond just extracting part of the cell. For example, this query returns one column with just the week in the year (between 1 and 53) for each date in the `Date` column: 

![](https://i.imgur.com/A5hqGxY.png)

You can find all the functions you can use with dates in BigQuery in [this documentation](https://cloud.google.com/bigquery/docs/reference/legacy-sql#datetimefunctions) under "Date and time functions".  

# Example: Which day of the week has the most fatal motor accidents?

Let's use the US Traffic Fatality Records database, which contains information on traffic accidents in the US where at least one person died.

We'll investigate the `accident_2015` table. Here is a view of the first few rows.  (_We have hidden the corresponding code. To take a peek, click on the "Code" button below._)

In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "nhtsa_traffic_fatalities" dataset
dataset_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "accident_2015" table
table_ref = dataset_ref.table("accident_2015")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "accident_2015" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,county,city,day_of_crash,month_of_crash,year_of_crash,day_of_week,hour_of_crash,minute_of_crash,national_highway_system,land_use,land_use_name,functional_system,functional_system_name,ownership,ownership_name,route_signing,route_signing_name,trafficway_identifier,trafficway_identifier_2,milepoint,latitude,longitude,special_jurisdiction,special_jurisdiction_name,first_harmful_event,first_harmful_event_name,manner_of_collision,manner_of_collision_name,relation_to_junction_within_interchange_area,relation_to_junction_specific_location,relation_to_junction_specific_location_name,type_of_intersection,work_zone,relation_to_trafficway,relation_to_trafficway_name,light_condition,light_condition_name,atmospheric_conditions_1,atmospheric_conditions_1_name,atmospheric_conditions_2,atmospheric_conditions_2_name,atmospheric_conditions,atmospheric_conditions_name,school_bus_related,rail_grade_crossing_identifier,hour_of_notification,minute_of_notification,hour_of_arrival_at_scene,minute_of_arrival_at_scene,hour_of_ems_arrival_at_hospital,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,8,Colorado,80038,1,1,0,0,0,4,4,63,0,4,2,2015,4,7,35,1,1,Rural,1,Interstate,1,State Highway Agency,1,Interstate,I-70,,4322,39.294631,-102.532778,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,3,On Median,1,Daylight,4,Snow,10,Cloudy,4,Snow,No,0000000,99,99,99,99,99,99,0,None,0,None,0,None,1,0,2015-02-04 07:35:00+00:00
1,51,Virginia,510048,1,1,0,0,0,1,1,740,1970,8,2,2015,1,2,40,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-264,,35,36.814722,-76.347842,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,4,On Roadside,3,Dark – Lighted,1,Clear,10,Cloudy,10,Cloudy,No,0000000,99,99,99,99,88,88,0,None,0,None,0,None,1,0,2015-02-08 02:40:00+00:00
2,51,Virginia,510157,2,2,0,0,0,2,2,710,1760,15,4,2015,4,6,45,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-64,,2850,36.832867,-76.195911,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,1,On Roadway,1,Daylight,1,Clear,10,Cloudy,10,Cloudy,No,0000000,6,47,6,59,88,88,0,None,0,None,0,None,1,0,2015-04-15 06:45:00+00:00
3,51,Virginia,510131,3,3,0,0,0,3,3,710,1760,2,4,2015,5,5,51,1,2,Urban,1,Interstate,1,State Highway Agency,1,Interstate,I-64,,2775,36.912778,-76.263056,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,1,On Roadway,3,Dark – Lighted,1,Clear,10,Cloudy,10,Cloudy,No,0000000,99,99,99,99,99,99,14,Motor Vehicle (In Transport 1983-2004) Struck ...,0,None,0,None,1,1,2015-04-02 05:51:00+00:00
4,56,Wyoming,560003,1,1,0,0,0,5,5,21,0,4,1,2015,1,8,32,1,1,Rural,1,Interstate,1,State Highway Agency,1,Interstate,I-25,,341,41.471800,-104.893100,0,No Special Jurisdiction (Includes National For...,1,Rollover/Overturn,0,Not Collision with Motor Vehicle in Transport ...,No,1,Non-Junction,Not an Intersection,None,3,On Median,1,Daylight,11,Blowing Snow,10,Cloudy,11,Blowing Snow,No,0000000,8,34,8,57,10,5,0,None,0,None,0,None,1,0,2015

Let's use the table to determine how the number of accidents varies with the day of the week.  Since:
- the `consecutive_number` column contains a unique ID for each accident, and
- the `timestamp_of_crash` column contains the date of the accident in DATETIME format,

we can:
- **EXTRACT** the day of the week (as `day_of_week` in the query below) from the `timestamp_of_crash` column, and
- **GROUP BY** the day of the week, before we **COUNT** the `consecutive_number` column to determine the number of accidents for each day of the week.

Then we sort the table with an **ORDER BY** clause, so the days with the most accidents are returned first.

In [2]:
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

As usual, we run it as follows:

In [3]:
# Set up the query
query_job = client.query(query)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


Notice that the data is sorted by the `num_accidents` column, where the days with more traffic accidents appear first.

To map the numbers returned for the `day_of_week` column to the actual day, you might consult [the BigQuery documentation](https://cloud.google.com/bigquery/docs/reference/legacy-sql#dayofweek) on the DAYOFWEEK function. It says that it returns "an integer between 1 (Sunday) and 7 (Saturday), inclusively". So, in 2015, most fatal motor accidents in the US occured on Sunday and Saturday, while the fewest happened on Tuesday.

# Your Turn
**ORDER BY** can make your results easier to interpret. **[Try it yourself](#$NEXT_NOTEBOOK_URL$)**.
